In [ ]:
import cmath
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import matplotlib.pyplot as plt

k=cmath.sqrt(-1)
Vb=220e+3  #V
Sb=100e+6  # VA
Zb=(Vb**2)/Sb #Ohm

ZL= 0.099 + 0.156*k

Z23=(ZL*10)/Zb  
Y23=1/Z23

Z72=(ZL*10)/Zb  
Y72=1/Z72

Z34=(ZL*15)/Zb  
Y34=1/Z34

Z45=(ZL*20)/Zb  
Y45=1/Z45

Z67=(ZL*30)/Zb  
Y67=1/Z67

Z56=(ZL*30)/Zb  
Y56=1/Z56

Y12= 1

Y11=Y12
Y22=Y12+Y23+Y72
Y33= Y23+Y34
Y44= Y34+Y45
Y55= Y45+Y56
Y66= Y56+Y67
Y77= Y67+Y72

Ybus=np.array([
    [Y11, -Y12, 0, 0, 0, 0, 0],
    [-Y12, Y22, -Y23, 0, 0, 0, -Y72],
    [0, -Y23, Y33, -Y34, 0, 0, 0],
    [0, 0 , -Y34, Y44, -Y45, 0, 0],
    [0, 0, 0, -Y45, Y55, -Y56, 0],
    [0, 0, 0, 0, -Y56, Y66, -Y67],
    [0, -Y72, 0, 0, 0, -Y67, Y77]])

print (Ybus)


# Definir los parámetros y variables iniciales
V = np.ones(len(Ybus))  # Vector de tensiones iniciales (modulo)
theta = np.zeros(len(Ybus))  # Vector de ángulos de fase iniciales (en radianes)
max_iter = 100  # Máximo número de iteraciones
tolerance = 1e-3  # Tolerancia para la convergencia

# Definir las funciones de cálculo de potencia
def calculate_power_mismatch(V, theta, Ybus, S_spec):
    P_calc = np.zeros(len(V))
    Q_calc = np.zeros(len(V))
    for i in range(len(V)):
        for j in range(len(V)):
            P_calc[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
            Q_calc[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    P_spec = S_spec.real
    Q_spec = S_spec.imag
    delta_P = P_spec - P_calc
    delta_Q = Q_spec - Q_calc
    return delta_P, delta_Q

# Definir las funciones de cálculo de la matriz Jacobiana
def calculate_jacobian(V, theta, Ybus):
    n = len(V)
    J = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            if i != j:
                J[i, j] = -V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
                J[i, j+n] = V[i] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[i+n, j] = V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[i+n, j+n] = -V[i] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
            else:
                for k in range(n):
                    if k != i:
                        J[i, j] += V[i] * V[k] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                        J[i, j+n] += V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J[i+n, j] += V[i] * V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J[i+n, j+n] += V[i] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                J[i, j] += V[i]**2 * Ybus[i, i].imag
                J[i+n, j+n] -= V[i]**2 * Ybus[i, i].imag
    return J

# Vector de potencias especificadas
S_spec = np.array([complex(Ybus[i, 2], Ybus[i, 3]) for i in range(len(Ybus))])

# Iteración de Newton-Raphson
for iter in range(max_iter):
    # Calcular desajustes de potencia
    delta_P, delta_Q = calculate_power_mismatch(V, theta, Ybus, S_spec)
    
    # Verificar convergencia
    if np.all(np.abs(delta_P) < tolerance) and np.all(np.abs(delta_Q) < tolerance):
        print(f"Convergencia alcanzada en {iter+1} iteraciones.")
        break
    
    # Calcular la matriz Jacobiana
    J = calculate_jacobian(V, theta, Ybus)
    
    # Crear el vector de desajustes
    delta_S = np.concatenate((delta_P, delta_Q))
    
    # Resolver el sistema de ecuaciones lineales
    delta_X = np.linalg.solve(J, delta_S)
    
    # Actualizar variables de estado
    theta += delta_X[:len(V)]
    V += delta_X[len(V):]
else:
    print("El algoritmo de Newton-Raphson no convergió en el número máximo de iteraciones.")

# Mostrar los resultados finales
print("Tensiones (modulos):", V)
print("Ángulos de fase (radianes):", theta)





In [ ]:
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x))


# Crear la red a analizar
red2 = pp.create_empty_network()

# Creación de las barras del sistema
barra_1 = pp.create_bus(red2, vn_kv=110, name="Barra 1")
barra_2 = pp.create_bus(red2, vn_kv=220, name="Barra 2")
barra_1A = pp.create_bus(red2, vn_kv=220, name="Barra 1A")
barra_1B = pp.create_bus(red2, vn_kv=220, name="Barra 1B")
barra_2A = pp.create_bus(red2, vn_kv=220, name="Barra 2A")
barra_2B = pp.create_bus(red2, vn_kv=220, name="Barra 2B")
barra_3A = pp.create_bus(red2, vn_kv=220, name="Barra 3A")


pp.create_transformer(red2, barra_2, barra_1, std_type="100 MVA 220/110 kV")

# Añadir el generador en la barra 1 y que dicha barra sea la referencia
pp.create_ext_grid(red2, barra_1, va_pu=1.0, name="Slack")

# Crear las líneas
pp.create_line(red2, barra_2, barra_1A, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(red2, barra_2, barra_1B, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1B")
pp.create_line(red2, barra_1A, barra_2A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(red2, barra_2A, barra_3A, length_km=20.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(red2, barra_1B, barra_2B, length_km=30.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1B-2B")
pp.create_line(red2, barra_2B, barra_3A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-3A")

# Crear las cargas
pp.create_load(red2, barra_1A, p_mw=30.0, q_mvar=20.0, name="Carga Barra 1A")
pp.create_load(red2, barra_2A, p_mw=52.5, q_mvar=35.0, name="Carga Barra 2A")
pp.create_load(red2, barra_3A, p_mw=22.5, q_mvar=15.0, name="Carga Barra 3A")
pp.create_load(red2, barra_1B, p_mw=15.0, q_mvar=10.0, name="Carga Barra 1B")
pp.create_load(red2, barra_2B, p_mw=90.0, q_mvar=60.0, name="Carga Barra 2B")

print(red2.line)

pp.runpp(red2)
from pandapower.converter import to_ppc
from pandapower.pypower.makeYbus import makeYbus

ppc = pp.converter.to_ppc(red2)

ppc = to_ppc(red2)

# Obtener los datos de los buses y ramas
baseMVA = ppc["baseMVA"]
bus = ppc["bus"]
branch = ppc["branch"]

# Crear la matriz Y-bus
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

# Convertir la matriz Y-bus a formato denso
Ybus_dense = np.array(Ybus.todense())

# Convertir la matriz densa a una matriz de NumPy
Ybus_np = np.array(Ybus_dense)

# Aproximar los términos de la matriz a 3 decimales
Ybus_rounded = np.round(Ybus_np, decimals=3)

# Mostrar la matriz Y-bus densa en formato NumPy, aproximada a 3 decimales
print("Matriz Y-bus (NumPy) aproximada a 3 decimales:")
print(Ybus_rounded)

# Coordenadas del término deseado (fila, columna)
f = 0
c = 0

# Extraer el término de la matriz Y-bus
termino = Ybus_rounded[f,c]

# Mostrar el término extraído
print(f"Término en la posición ({f}, {c}): {termino}")

# Definir los parámetros y variables iniciales
V = np.ones(len(bus))  # Vector de tensiones iniciales (modulo)
theta = np.zeros(len(bus))  # Vector de ángulos de fase iniciales (en radianes)
max_iter = 100  # Máximo número de iteraciones
tolerance = 1e-3  # Tolerancia para la convergencia

# Definir las funciones de cálculo de potencia
def calculate_power_mismatch(V, theta, Ybus, S_spec):
    P_calc = np.zeros(len(V))
    Q_calc = np.zeros(len(V))
    for i in range(len(V)):
        for j in range(len(V)):
            P_calc[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
            Q_calc[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    P_spec = S_spec.real
    Q_spec = S_spec.imag
    delta_P = P_spec - P_calc
    delta_Q = Q_spec - Q_calc
    return delta_P, delta_Q

# Definir las funciones de cálculo de la matriz Jacobiana
def calculate_jacobian(V, theta, Ybus):
    n = len(V)
    J = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            if i != j:
                J[i, j] = -V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
                J[i, j+n] = V[i] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[i+n, j] = V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[i+n, j+n] = -V[i] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
            else:
                for k in range(n):
                    if k != i:
                        J[i, j] += V[i] * V[k] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                        J[i, j+n] += V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J[i+n, j] += V[i] * V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J[i+n, j+n] += V[i] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                J[i, j] += V[i]**2 * Ybus[i, i].imag
                J[i+n, j+n] -= V[i]**2 * Ybus[i, i].imag
    return J

# Vector de potencias especificadas
S_spec = np.array([complex(bus[i, 2], bus[i, 3]) for i in range(len(bus))]) / baseMVA

# Iteración de Newton-Raphson
for iter in range(max_iter):
    # Calcular desajustes de potencia
    delta_P, delta_Q = calculate_power_mismatch(V, theta, Ybus_np, S_spec)
    
    # Verificar convergencia
    if np.all(np.abs(delta_P) < tolerance) and np.all(np.abs(delta_Q) < tolerance):
        print(f"Convergencia alcanzada en {iter+1} iteraciones.")
        break
    
    # Calcular la matriz Jacobiana
    J = calculate_jacobian(V, theta, Ybus_np)
    
    # Crear el vector de desajustes
    delta_S = np.concatenate((delta_P, delta_Q))
    
    # Resolver el sistema de ecuaciones lineales
    delta_X = np.linalg.solve(J, delta_S)
    
    # Actualizar variables de estado
    theta += delta_X[:len(V)]
    V += delta_X[len(V):]
else:
    print("El algoritmo de Newton-Raphson no convergió en el número máximo de iteraciones.")

# Mostrar los resultados finales
print("Tensiones (modulos):", V)
print("Ángulos de fase (radianes):", theta)



